In [74]:
import requests
import json
import pandas as pd
import time
import numpy as np
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 10.0)

## Functions

In [75]:
def extract_data(data, device_num, start_date, end_date, parameter):
    column_name = '%s_%s' % (device_num,parameter)
    columns_to_select = ['UTC time', column_name]
    data_for_device = data[columns_to_select]
    data_for_device = data_for_device[data_for_device['UTC time'] >= start_date]
    data_for_device = data_for_device[data_for_device['UTC time'] <= end_date]
    return data_for_device

## Read airly data from 2018

In [76]:
locations = pd.read_csv("data/location_const.csv")
locations = locations[['id', 'latitude', 'longitude', 'city', 'street', 'number','elevation']]
locations = locations.set_index('id', drop = False)
airly_data_2018 = pd.read_csv("data/data_from_airly_api.csv")

start_date = "2018-10-23T00:00:00"
end_date = "2019-01-07T23:00:00"
parameter = 'pm25'
for loc in locations['id']:
    try:
        data = extract_data(airly_data_2018, loc, start_date, end_date, parameter)
        change = data['%s_%s' % (loc,parameter)]
        locations.loc[loc, 'values'] = ','.join(list(map(lambda x: str(x), change)))
        locations.loc[loc, 'trend'] = ','.join(list(map(lambda x: str(x), np.diff(change))))
    except KeyError:
        locations = locations.drop(index=loc)
        continue
locations

,id,latitude,longitude,city,street,number,elevation,values,trend
id,,,,,,,,,
820,820,50.052433,19.949579,Kraków,Starowiślna,79a,207.02,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na..."
1081,1081,50.055163,19.947015,Kraków,Starowiślna,38,207.89,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na..."
622,622,50.044424,19.952205,Kraków,Krakusa,11,205.57,"12.63,12.7,15.86,18.88,23.66,29.35,24.09,17.02...","0.06999999999999851,3.16,3.0199999999999996,4...."
842,842,50.042480,19.944330,Kraków,Kalwaryjska,30,204.47,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na..."
2277,2277,50.056538,19.954229,Kraków,Prochowa,1,203.75,"11.41,10.51,14.97,18.23,21.48,25.77,23.09,15.2...","-0.9000000000000004,4.460000000000001,3.26,3.2..."
201,201,50.054907,19.956501,Kraków,Masarska,9,203.13,"10.06,10.09,12.97,15.2,19.71,23.64,21.75,14.42...","0.02999999999999936,2.880000000000001,2.229999..."
1026,1026,50.038499,19.946850,Kraków,Krzemionki,30,240.90,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na..."
2701,2701,50.056919,19.929686,Kraków,Felicjanek,18,208.45,"12.29,12.63,14.84,19.2,21.69,27.18,24.42,17.8,...","0.34000000000000163,2.209999999999999,4.359999..."
204,204,50.062006,19.940984,Kraków,Mikołajska,4B,220.38,"9.41,8.7,11.93,13.11,16.84,23.01,21.29,14.32,1...","-0.7100000000000009,3.2300000000000004,1.17999..."


## Prepare triangulation with average values for each triangle

In [77]:
def get_avg_data_for_triangle(triangle, values):
    values_for_triangle = list(np.array(values)[triangle])
    values_for_triangle = [x.split(',') for x in values_for_triangle]
    return [np.nanmean(np.array(x).astype(np.float))for x in list(zip(*values_for_triangle))]

In [78]:
points = [[float(x), float(y)] for x,y in zip(list(locations['longitude']), list(locations['latitude']))]
points = np.array(points)
tri = Delaunay(points)
triangles = tri.simplices

values = list(locations['values'])
trend = list(locations['trend'])
avg = []
avg_trend = []
for triangle in triangles:
    avg.append(get_avg_data_for_triangle(triangle, values))
    avg_trend.append(get_avg_data_for_triangle(triangle, trend))
print(avg)


/home/beata/.local/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Create correlation matrix

In [79]:
from scipy.cluster.hierarchy import linkage
from  scipy.cluster.hierarchy import fcluster

corr_matrix = np.nan_to_num(np.corrcoef(avg))
dissimilarity = 1 - np.abs(corr_matrix)
hierarchy = linkage(dissimilarity, method='average')
labels = fcluster(hierarchy, 0.5, criterion='distance')

print(labels)

[ 1  1 25  1  1  1  1  1  1  1  1  1  1 25 19  1  1  1  1 15  1  1  1 15
 19 16 16 19 19 19 12 15  8 25  1  1  6  2  1  3  1  1  1  1  1  1  1  1
  1  1  1 19  1 15  1 12 12  8  1  1  1 11 14 14 14  1  7  7  1  7  1  1
  1 22  1 16 22 24 25 24  1  1  1  1  1  1  1  1 20  1  1 22 16 16 15 19
  1  1  1  1  1  1  8  7  1 11  1 15 15  1  1  1  1  1  1  1  5 22 22  8
  7  1 24 25 25  1 24 24 24 24 24 24 24 12 12 12 21 21  1 21 22 14 14 22
 24 24  1 24  1 25 27  1  6  1  1 19 19  1  1  1 23 23 26 22 24  1  1  1
  1  1  1  1  1  1  1 22 22 22  8  8  8 14 13  1  4  1  1  4  4  1  1  1
  1  1  1 14 21 22  1 22 22 22 18 18 21 21 21  1 22  1 22  1 24  1 27 27
 26  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 27  1  1  1 18 18 18
  1 25  1  1  1  1  1  1  1  1 23 23 23 26  1  1  1 26 26 22 22 21 22 22
 22 22 22 18 22 22  1  1  1  1  1  5  5  1  5 22 22 22 21 21 22  1 24 24
 24  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 27  1  1 22 22 22 22 22
 22  1  1  1  1  1  1  1  1  1  1 18 22  1  1  1  1

In [80]:
corr_matrix = np.nan_to_num(np.corrcoef(avg_trend))
dissimilarity_trend = 1 - np.abs(corr_matrix)
hierarchy = linkage(dissimilarity_trend, method='average')
labels_trend = fcluster(hierarchy, 0.5, criterion='distance')
print(labels_trend)


[  1   1  54   1   1   1   1   1   1   1   1   1   1   1 129   1   1   1
   1   1   1   1   1  99  68  67  67   1  64  65  78  59  84   1   1   1
  56  57   1  58   1   1   1   1   1   1   1   1   1   1   1  69   1  59
   1   1   1   1   1   1   1  62 101 103 102   1  78  82   1   1   1   1
   1   1   1 100  26 119 112 116   1   1   1   1   1   1   1   1 127   1
   1  26  66  66  60   1   1   1   1   1   1   1  70  83   1  63   1  61
  60   1   1   1   1   1   1   1  76   9   9  74  81   1 113 111 111   1
 114 115 126 125 123 124 125  89  87  88  94  94   1  95  37  92  93  22
 121 122   1 121   1 117 137   1  55   1   1 130 128   1   1   1 149 151
 150  14 119   1   1   1   1   1   1   1   1   1   1   6   8   8  71  79
  80  85  86   1  77   1   1  72  72   1   1   1   1   1   1  92  96  10
   1  23  11  21  43  41  96  90  91   1  37   1   4   1 120   1 136 136
 139   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 138
   1   1   1  39  34  35   1 118   1   1   1   1   

## Prepare GeoJSON file for correlation clusters (pm2.5)

In [81]:
data = {
     "type": "FeatureCollection",
     "features": []
}
for idx, triangle in enumerate(triangles):  
    data['features'].append({
                "type" : "Feature",
                "id" : idx,
                "geometry" : {
                    "type" : "Polygon",
                    "coordinates" : [points[triangle].tolist()]
                }
            })

with open('data/clusters_tendencies_pm25.json', 'w') as outfile:  
    json.dump(data, outfile)

In [82]:
avg_avg = np.array([ np.mean(row)  for row in np.nan_to_num(avg)])
round_avg = [int(round(x)) for x in avg_avg]

avg_avg_trend = np.array([ np.mean(row)  for row in np.nan_to_num(avg_trend)])
round_avg_trend = [int(round(x)) for x in avg_avg_trend]


## Plot pm 2.5 per region

In [83]:
import folium
import os
import bokeh.palettes
import branca


color = bokeh.palettes.viridis(max(labels))
color_avg = bokeh.palettes.viridis(max(round_avg) - min(round_avg) + 1)

color_trend = bokeh.palettes.viridis(max(labels_trend))
color_avg_trend = bokeh.palettes.viridis(max(round_avg_trend) - min(round_avg_trend) + 1)

m = folium.Map(
    location=[50.052433, 19.949579],
    zoom_start=10,
    tiles='openstreetmap'
)

colormap = branca.colormap.LinearColormap(colors=color_avg, index=list(range(min(round_avg), max(round_avg))),vmin=min(round_avg),vmax=max(round_avg))
colormap.add_to(m)


tri_json = os.path.join('data', 'clusters_tendencies_pm25.json')

folium.GeoJson(tri_json,  name='Correlations clusters (change tendences for pm 2.5)', style_function=lambda x :{'fillColor': color_trend[labels_trend[x['id']] - 1], 'fillOpacity': 0.7, 'weight': 0.5, 'color': 'black'}).add_to(m)
folium.GeoJson(tri_json,  show=False, name='Correlations clusters (values for pm 2.5)', style_function=lambda x :{'fillColor': color[labels[x['id']] - 1], 'fillOpacity': 0.7, 'weight': 0.5, 'color': 'black'}).add_to(m)
folium.GeoJson(tri_json, show=False, name='Avarage value per region (pm 2.5)', style_function=lambda x :{'fillColor': color_avg[round_avg[x['id']] - min(round_avg)], 'fillOpacity': 0.7, 'weight': 0.5, 'color': 'black'}).add_to(m)

folium.LayerControl().add_to(m)
m

In [84]:
from folium.plugins import HeatMap
devices = folium.Map(
    location=[50.052433, 19.949579],
    zoom_start=10,
    tiles='openstreetmap'
)

values = list(locations['values'])
values = [x.split(',') for x in values]
values = [np.nanmean(np.array(x).astype(np.float))for x in list(zip(*values))]
points = [[float(x), float(y), int(round(avg))] for x,y,avg in zip(list(locations['latitude']), list(locations['longitude']), values)]

round_avg = [point[2] for point in points]
color = bokeh.palettes.viridis(max(round_avg))
color_avg = bokeh.palettes.viridis(max(round_avg) - min(round_avg) + 1)


colormap = branca.colormap.LinearColormap(colors=color_avg, index=list(range(min(round_avg), max(round_avg))),vmin=min(round_avg),vmax=max(round_avg))
colormap.add_to(devices)


for point in points:
    folium.CircleMarker(
    location=[point[0], point[1]],
    radius=30,
    color=color_avg[point[2] - min(round_avg)],
    fill=True,
    stroke=False,
    fill_color=color_avg[point[2] - min(round_avg)],
).add_to(devices)

devices

## Clustering quality measure

In [85]:
from sklearn import metrics

silhouette_score = metrics.silhouette_score(dissimilarity, labels, metric='euclidean')
calinski_harabaz_score = metrics.calinski_harabaz_score(dissimilarity, labels)  
davies_bouldin_score = metrics.davies_bouldin_score(dissimilarity, labels)  
print(silhouette_score, calinski_harabaz_score, davies_bouldin_score)

(0.6581301599343283, 23061.59733918957, 0.7389364309543841)


In [86]:
silhouette_score = metrics.silhouette_score(dissimilarity_trend, labels_trend, metric='euclidean')
calinski_harabaz_score = metrics.calinski_harabaz_score(dissimilarity_trend, labels_trend)  
davies_bouldin_score = metrics.davies_bouldin_score(dissimilarity_trend, labels_trend)  
print(silhouette_score, calinski_harabaz_score, davies_bouldin_score)

(0.6286847142284661, 1881.5047199404908, 0.34098264999720335)
